#### Start with installing the required library

In [3]:
!pip install snowflake-connector-python[pandas]

  Using cached snowflake_connector_python-2.4.3-cp36-cp36m-manylinux2014_x86_64.whl (13.2 MB)
  Using cached azure_common-1.1.27-py2.py3-none-any.whl (12 kB)
  Using cached oscrypto-1.2.1-py2.py3-none-any.whl (192 kB)
  Using cached PyJWT-2.1.0-py3-none-any.whl (16 kB)
  Using cached pycryptodomex-3.10.1-cp35-abi3-manylinux2010_x86_64.whl (1.9 MB)
  Using cached azure_storage_blob-12.8.1-py2.py3-none-any.whl (345 kB)
  Using cached pyarrow-3.0.0-cp36-cp36m-manylinux2014_x86_64.whl (20.7 MB)
  Using cached azure_core-1.14.0-py2.py3-none-any.whl (136 kB)
  Using cached msrest-0.6.21-py2.py3-none-any.whl (85 kB)
  Using cached requests_oauthlib-1.3.0-py2.py3-none-any.whl (23 kB)
  Using cached isodate-0.6.0-py2.py3-none-any.whl (45 kB)
  Using cached oauthlib-3.1.0-py2.py3-none-any.whl (147 kB)
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 4.0.0
    Uninstalling pyarrow-4.0.0:
      Successfully uninstalled pyarrow-4.0.0


### Import the required Packages

In [4]:
import pandas as pd
from snowflake.connector import pandas_tools as sfpd
import snowflake.connector
import toml

In [5]:
df = pd.read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/abalone/abalone.data",
                names = ['sex', 'length', 'diameter','height','whole_weight','shucked_weight',
                           'viscera_weight','shell_weight','rings'])

In [24]:
df.head()

,SEX,LENGTH,DIAMETER,HEIGHT,WHOLE_WEIGHT,SHUCKED_WEIGHT,VISCERA_WEIGHT,SHELL_WEIGHT,RINGS
0,M,0.455,0.365,0.095,0.5140,0.2245,0.1010,0.150,15
1,M,0.350,0.265,0.090,0.2255,0.0995,0.0485,0.070,7
2,F,0.530,0.420,0.135,0.6770,0.2565,0.1415,0.210,9
3,M,0.440,0.365,0.125,0.5160,0.2155,0.1140,0.155,10
4,I,0.330,0.255,0.080,0.2050,0.0895,0.0395,0.055,7


#### Connect to Snowflake 

In [25]:
env = toml.load('config.toml')
conn = snowflake.connector.connect(
                user= env['trial'].get('user'),
                password= env['trial'].get('password'),
                account= env['trial'].get('account'),
                warehouse=env['trial'].get('warehouse'),
                database=env['trial'].get('database'),
                schema=env['trial'].get('schema')
                )

#### Create the Table into which we'll load the data

In [26]:
tbl = "ABALONE_TEST"
cur = conn.cursor()
cur.execute(f'drop table if exists "SGDB"."PUBLIC".{tbl};')
cur.execute(f'''CREATE TABLE "SGDB"."PUBLIC".{tbl} ("SEX" STRING, "LENGTH" NUMBER (5, 4), 
            "DIAMETER" NUMBER (5, 4), "HEIGHT" NUMBER (5, 4), "WHOLE_WEIGHT" NUMBER (5, 4), 
            "SHUCKED_WEIGHT" NUMBER (5, 4), "VISCERA_WEIGHT" NUMBER (5, 4), "SHELL_WEIGHT" NUMBER (5, 4),
            "RINGS" INTEGER);''');

In [27]:
df.columns = [i.upper() for i in df.columns] #update to help with import into snowflake

Load the Data and Check if the process succeeded

In [28]:
success, nchunks, nrows, _ = sfpd.write_pandas(conn, df,tbl) # write to snowflake

print(f"INFO: Write Pandas method reported Success = {success}, with a total number of {nrows} rows written to the table")

INFO: Write Pandas method reported Success = True, with a total number of 4177 rows written to the table


In [29]:
cur.close()
conn.close()